질의응답
질문이 키워드로 구성됨
answer span을 찾는 행위.
intructive QA. 요새는 검색까지 포함.
예전에는 reader모델. 현재는 방대한 문서에서 찾으려고, open domain QA로 나아감.

In [2]:
!pip install git+https://github.com/huggingface/transformers.git
!pip install datasets
!pip install huggingface-hub

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
  Cloning https://github.com/huggingface/transformers.git to /tmp/pip-req-build-z8xjebbt
  Running command git clone -q https://github.com/huggingface/transformers.git /tmp/pip-req-build-z8xjebbt
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
    Preparing wheel metadata ... done
     |████████████████████████████████| 163 kB 4.7 MB/s 
     |████████████████████████████████| 7.6 MB 30.3 MB/s 
  Created wheel for transformers: filename=transformers-4.25.0.dev0-py3-none-any.whl size=5526118 sha256=03a1a13d868270a6d57dd4c21ecbd0a381d2ff2fefea92a8245c14af3a502092
  Stored in directory: /tmp/pip-ephem-wheel-cache-69yzob3z/wheels/90/a5/44/6bcd83827c8a60628c5ad602f429cd5076bcce5f2a90054947
Successfully built transformers
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |███████████████████████████████

In [3]:
from datasets import load_dataset

datasets = load_dataset("squad")

Extracting data files:   0%|          | 0/2 [00:00<?, ?it/s]

Generating train split:   0%|          | 0/87599 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/10570 [00:00<?, ? examples/s]

Dataset squad downloaded and prepared to /root/.cache/huggingface/datasets/squad/plain_text/1.0.0/d6ec3ceb99ca480ce37cdd35555d6cb2511d223b9150cce08a837ef62ffea453. Subsequent calls will reuse this data.


  0%|          | 0/2 [00:00<?, ?it/s]

In [4]:
print(datasets["train"][0])

{'id': '5733be284776f41900661182', 'title': 'University_of_Notre_Dame', 'context': 'Architecturally, the school has a Catholic character. Atop the Main Building\'s gold dome is a golden statue of the Virgin Mary. Immediately in front of the Main Building and facing it, is a copper statue of Christ with arms upraised with the legend "Venite Ad Me Omnes". Next to the Main Building is the Basilica of the Sacred Heart. Immediately behind the basilica is the Grotto, a Marian place of prayer and reflection. It is a replica of the grotto at Lourdes, France where the Virgin Mary reputedly appeared to Saint Bernadette Soubirous in 1858. At the end of the main drive (and in a direct line that connects through 3 statues and the Gold Dome), is a simple, modern stone statue of Mary.', 'question': 'To whom did the Virgin Mary allegedly appear in 1858 in Lourdes France?', 'answers': {'text': ['Saint Bernadette Soubirous'], 'answer_start': [515]}}


In [5]:
datasets

DatasetDict({
    train: Dataset({
        features: ['id', 'title', 'context', 'question', 'answers'],
        num_rows: 87599
    })
    validation: Dataset({
        features: ['id', 'title', 'context', 'question', 'answers'],
        num_rows: 10570
    })
})

In [6]:
datasets.keys()

dict_keys(['train', 'validation'])

In [7]:
datasets['train']

Dataset({
    features: ['id', 'title', 'context', 'question', 'answers'],
    num_rows: 87599
})

In [8]:
datasets['train'][0]

{'id': '5733be284776f41900661182',
 'title': 'University_of_Notre_Dame',
 'context': 'Architecturally, the school has a Catholic character. Atop the Main Building\'s gold dome is a golden statue of the Virgin Mary. Immediately in front of the Main Building and facing it, is a copper statue of Christ with arms upraised with the legend "Venite Ad Me Omnes". Next to the Main Building is the Basilica of the Sacred Heart. Immediately behind the basilica is the Grotto, a Marian place of prayer and reflection. It is a replica of the grotto at Lourdes, France where the Virgin Mary reputedly appeared to Saint Bernadette Soubirous in 1858. At the end of the main drive (and in a direct line that connects through 3 statues and the Gold Dome), is a simple, modern stone statue of Mary.',
 'question': 'To whom did the Virgin Mary allegedly appear in 1858 in Lourdes France?',
 'answers': {'text': ['Saint Bernadette Soubirous'], 'answer_start': [515]}}

In [9]:
datasets['train'][0]['question']

'To whom did the Virgin Mary allegedly appear in 1858 in Lourdes France?'

In [10]:
datasets['train'][0]['answers'] # answer_start = 답이 있는 시작지점. 여러개라면 답이 리스트로 표현될 것이다.
# 답 span을어디 기준으로 맞춰야하는지가 어려운 부분이다.

# context, question, answer가 이미 있으므로, 우리는 전처리만 하면 된다. 숫자화만 하면 된다.
# 토큰화를 하는 것, 결국엔.

{'text': ['Saint Bernadette Soubirous'], 'answer_start': [515]}

In [14]:
from transformers import AutoTokenizer, AutoConfig

model_checkpoint = "distilbert-base-cased"
# 모델 - base:사이즈 - case:대문자를 인식한 상태로 학습됬는가? 아니면 모두 소문자처리해서 했는가.

tokenizer = AutoTokenizer.from_pretrained(model_checkpoint)
#트랜스포머에서 가장 많이 쓰는게 이 오토토크나이저,
#+ AutoConfig

In [15]:
config = AutoConfig.from_pretrained("distilbert-base-cased")
config

DistilBertConfig {
  "_name_or_path": "distilbert-base-cased",
  "activation": "gelu",
  "attention_dropout": 0.1,
  "dim": 768,
  "dropout": 0.1,
  "hidden_dim": 3072,
  "initializer_range": 0.02,
  "max_position_embeddings": 512,
  "model_type": "distilbert",
  "n_heads": 12,
  "n_layers": 6,
  "output_past": true,
  "pad_token_id": 0,
  "qa_dropout": 0.1,
  "seq_classif_dropout": 0.2,
  "sinusoidal_pos_embds": false,
  "tie_weights_": true,
  "transformers_version": "4.25.0.dev0",
  "vocab_size": 28996
}

In [16]:
config = AutoConfig.from_pretrained("bert-base-uncased")
config

Downloading:   0%|          | 0.00/570 [00:00<?, ?B/s]

BertConfig {
  "_name_or_path": "bert-base-uncased",
  "architectures": [
    "BertForMaskedLM"
  ],
  "attention_probs_dropout_prob": 0.1,
  "classifier_dropout": null,
  "gradient_checkpointing": false,
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.1,
  "hidden_size": 768,
  "initializer_range": 0.02,
  "intermediate_size": 3072,
  "layer_norm_eps": 1e-12,
  "max_position_embeddings": 512,
  "model_type": "bert",
  "num_attention_heads": 12,
  "num_hidden_layers": 12,
  "pad_token_id": 0,
  "position_embedding_type": "absolute",
  "transformers_version": "4.25.0.dev0",
  "type_vocab_size": 2,
  "use_cache": true,
  "vocab_size": 30522
}

In [17]:
# 풀고싶은 단락이 있으면 무작정 넣지 않고, 최대길이를 지정해줘야 한다.
# doc_stride: 단락이 너무 길면 좀 잘라서 보는 테크닉.
# doc이 너무 길면 ... 근데 쪼개지는 부분에 답이 존재하면 문맥이 끊기니까, 300 = 100 100 100 100 100 으로, 좀 겹치는 부분이 있게 나눈다.

In [26]:
max_length = 384  # The maximum length of a feature (question and context)
doc_stride = (
    128  # The authorized overlap between two part of the context when splitting
)
# it is needed.

In [25]:
# 전처리르해서 질문과 컨텍스트가 있을 때, 숫자화를 잘 시켜서 모델에 적용시켜 줄 수 있는 모델.
def prepare_train_features(examples):
    # Tokenize our examples with truncation and padding, but keep the overflows using a
    # stride. This results in one example possible giving several features when a context is long,
    # each of those features having a context that overlaps a bit the context of the previous
    # feature.
    examples["question"] = [q.lstrip() for q in examples["question"]]
    examples["context"] = [c.lstrip() for c in examples["context"]]
    tokenized_examples = tokenizer(
        examples["question"],
        examples["context"],
        truncation="only_second",
        max_length=max_length,
        stride=doc_stride,
        return_overflowing_tokens=True,
        return_offsets_mapping=True,
        padding="max_length",
    )

    # Since one example might give us several features if it has a long context, we need a
    # map from a feature to its corresponding example. This key gives us just that.
    sample_mapping = tokenized_examples.pop("overflow_to_sample_mapping")
    # The offset mappings will give us a map from token to character position in the original
    # context. This will help us compute the start_positions and end_positions.
    offset_mapping = tokenized_examples.pop("offset_mapping")

    # Let's label those examples!
    tokenized_examples["start_positions"] = []
    tokenized_examples["end_positions"] = []

    for i, offsets in enumerate(offset_mapping):
        # We will label impossible answers with the index of the CLS token.
        input_ids = tokenized_examples["input_ids"][i]
        cls_index = input_ids.index(tokenizer.cls_token_id)

        # Grab the sequence corresponding to that example (to know what is the context and what
        # is the question).
        sequence_ids = tokenized_examples.sequence_ids(i)

        # One example can give several spans, this is the index of the example containing this
        # span of text.
        sample_index = sample_mapping[i]
        answers = examples["answers"][sample_index]
        # If no answers are given, set the cls_index as answer.
        if len(answers["answer_start"]) == 0:
            tokenized_examples["start_positions"].append(cls_index)
            tokenized_examples["end_positions"].append(cls_index)
        else:
            # Start/end character index of the answer in the text.
            start_char = answers["answer_start"][0]
            end_char = start_char + len(answers["text"][0])

            # Start token index of the current span in the text.
            token_start_index = 0
            while sequence_ids[token_start_index] != 1:
                token_start_index += 1

            # End token index of the current span in the text.
            token_end_index = len(input_ids) - 1
            while sequence_ids[token_end_index] != 1:
                token_end_index -= 1

            # Detect if the answer is out of the span (in which case this feature is labeled with the
            # CLS index).
            if not (
                offsets[token_start_index][0] <= start_char
                and offsets[token_end_index][1] >= end_char
            ):
                tokenized_examples["start_positions"].append(cls_index)
                tokenized_examples["end_positions"].append(cls_index)
            else:
                # Otherwise move the token_start_index and token_end_index to the two ends of the
                # answer.
                # Note: we could go after the last offset if the answer is the last word (edge
                # case).
                while (
                    token_start_index < len(offsets)
                    and offsets[token_start_index][0] <= start_char
                ):
                    token_start_index += 1
                tokenized_examples["start_positions"].append(token_start_index - 1)
                while offsets[token_end_index][1] >= end_char:
                    token_end_index -= 1
                tokenized_examples["end_positions"].append(token_end_index + 1)

    return tokenized_examples


In [21]:
samples_q = datasets['train'][0]['question']
print(samples_q)
tokenizer(samples_q)['input_ids']

To whom did the Virgin Mary allegedly appear in 1858 in Lourdes France?


[101,
 1706,
 2292,
 1225,
 1103,
 6567,
 2090,
 9273,
 2845,
 1107,
 8109,
 1107,
 10111,
 20500,
 1699,
 136,
 102]

In [22]:
# 잘려나갔던 토큰이 아이디로 변환되었고, CLS, SEP 토큰을 제외한 나머지토큰들이, 질문에 대해 숫자화로 변경된 형태가 나온것이다.

In [23]:
# 토큰의 위치와 문장 내에서의 위치. index위치 조정(offset). 그래서 최종적으로 맞춘 토큰이 어디서부터 어디까지인가? 를 알수있도록도 설저오디어 잇음

In [27]:
tokenized_datasets = datasets.map(
    prepare_train_features,
    batched=True,
    remove_columns=datasets["train"].column_names,
    num_proc=3,
)

# 

#0:   0%|          | 0/30 [00:00<?, ?ba/s]

#1:   0%|          | 0/30 [00:00<?, ?ba/s]

#2:   0%|          | 0/30 [00:00<?, ?ba/s]

#0:   0%|          | 0/4 [00:00<?, ?ba/s]

#1:   0%|          | 0/4 [00:00<?, ?ba/s]

#2:   0%|          | 0/4 [00:00<?, ?ba/s]

In [ ]:
# 이 위 단계가 가장 중요하다~ 성능의 차이가 많이 날거다.

In [31]:
datasets['train']

Dataset({
    features: ['id', 'title', 'context', 'question', 'answers'],
    num_rows: 87599
})

In [30]:
tokenized_datasets['train'] # 이제는 모델에 넣기 쉽게 변형되있는것을 확인할 수 있다. 위 datsets에 비해.

Dataset({
    features: ['input_ids', 'attention_mask', 'start_positions', 'end_positions'],
    num_rows: 88729
})

In [33]:
sample_input = tokenized_datasets['train'][0]['input_ids']
[token for token in sample_input if token != 0]

[101,
 1706,
 2292,
 1225,
 1103,
 6567,
 2090,
 9273,
 2845,
 1107,
 8109,
 1107,
 10111,
 20500,
 1699,
 136,
 102,
 22182,
 1193,
 117,
 1103,
 1278,
 1144,
 170,
 2336,
 1959,
 119,
 1335,
 4184,
 1103,
 4304,
 4334,
 112,
 188,
 2284,
 10945,
 1110,
 170,
 5404,
 5921,
 1104,
 1103,
 6567,
 2090,
 119,
 13301,
 1107,
 1524,
 1104,
 1103,
 4304,
 4334,
 1105,
 4749,
 1122,
 117,
 1110,
 170,
 7335,
 5921,
 1104,
 4028,
 1114,
 1739,
 1146,
 14089,
 5591,
 1114,
 1103,
 7051,
 107,
 159,
 21462,
 1566,
 24930,
 2508,
 152,
 1306,
 3965,
 107,
 119,
 5893,
 1106,
 1103,
 4304,
 4334,
 1110,
 1103,
 19349,
 1104,
 1103,
 11373,
 4641,
 119,
 13301,
 1481,
 1103,
 171,
 17506,
 9538,
 1110,
 1103,
 144,
 10595,
 2430,
 117,
 170,
 14789,
 1282,
 1104,
 8070,
 1105,
 9284,
 119,
 1135,
 1110,
 170,
 16498,
 1104,
 1103,
 176,
 10595,
 2430,
 1120,
 10111,
 20500,
 117,
 1699,
 1187,
 1103,
 6567,
 2090,
 25153,
 1193,
 1691,
 1106,
 2216,
 17666,
 6397,
 3786,
 1573,
 25422,
 13149,
 11

In [36]:
# 그럼 답은 어떻게 줘야 하냐?

tokenized_datasets['train'][0]['start_positions']

136

In [37]:
train_set = tokenized_datasets["train"].with_format("numpy")[
    :
]  # Load the whole dataset as a dict of numpy arrays
validation_set = tokenized_datasets["validation"].with_format("numpy")[:]

## numpy 형태로 변환

In [38]:
from transformers import TFAutoModelForQuestionAnswering

model = TFAutoModelForQuestionAnswering.from_pretrained(model_checkpoint)
# 모델을 불러온다. 뼈대를 이미 만들어놓음.
# BERT 모델 위에 clasifiier 이미. 
# 사전에 하습된 파라미터를 뼈대에 넣는 과정이 마지막줄이다.
# 뜨는 메세지가 중요함. 뼈대가 사전학습보다 더 가질 수 있는데, 그거에 대한 얘기일 수 있음

Downloading:   0%|          | 0.00/354M [00:00<?, ?B/s]

Some layers from the model checkpoint at distilbert-base-cased were not used when initializing TFDistilBertForQuestionAnswering: ['activation_13', 'vocab_transform', 'vocab_layer_norm', 'vocab_projector']
- This IS expected if you are initializing TFDistilBertForQuestionAnswering from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing TFDistilBertForQuestionAnswering from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some layers of TFDistilBertForQuestionAnswering were not initialized from the model checkpoint at distilbert-base-cased and are newly initialized: ['qa_outputs', 'dropout_19']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [39]:
import tensorflow as tf
from tensorflow import keras

optimizer = keras.optimizers.Adam(learning_rate=5e-5)

In [40]:
# Optionally uncomment the next line for float16 training
keras.mixed_precision.set_global_policy("mixed_float16")

model.compile(optimizer=optimizer)

No loss specified in compile() - the model's internal loss computation will be used as the loss. Don't panic - this is a common way to train TensorFlow models in Transformers! To disable this behaviour please pass a loss argument, or explicitly pass `loss=None` if you do not want your model to compute a loss.


In [45]:
model.fit(train_set, validation_data=validation_set, epochs=1)
# 학습 엄청 오래걸림. 1 epoch에 1시간.
# 실시간으로 loss가 줄어드는 걸 볼 수 있는게 편하다.

 507/2773 [====>.........................] - ETA: 50:03 - loss: 2.2890

KeyboardInterrupt: ignored

In [ ]:
filepath = '/content/drive/MyDrive/keras_nlp_QA_example_1105.h5'

tf.keras.models.save_model(
    model,
    filepath,
    overwrite=True,
    include_optimizer=True,
    save_format=None,
    signatures=None,
    options=None,
    save_traces=True
)

In [47]:
# save and load
# ref: https://financial-engineering.medium.com/%EC%83%88%EB%A1%9C%EC%9A%B4-%ED%85%90%EC%84%9C%ED%94%8C%EB%A1%9C%EC%9A%B02-0-%EB%A7%8C%EB%93%A0-%EB%AA%A8%EB%8D%B8%EC%9D%84-%EC%A0%80%EC%9E%A5%ED%95%98%EA%B3%A0-%EB%B6%88%EB%9F%AC%EC%98%A4%EA%B8%B0-5da506b59e13
checkpoint_path = '/content/drive/MyDrive/keras_nlp_QA_example_1105.ckpt'
model.save_weights(checkpoint_path.format(epoch=0))


In [50]:
# load
import os
checkpoint_dir = os.path.dirname(checkpoint_path)
! ls {checkpoint_dir}
latest = tf.train.latest_checkpoint(checkpoint_dir)
latest

 ADHD_Project
 checkpoint
 Hanyang_Colab_Notebooks
 keras_nlp_QA_example_1105.ckpt.data-00000-of-00001
 keras_nlp_QA_example_1105.ckpt.index
 Pytorch_tutorial
'허깅페이스로 Question answering 진행하기.ipynb'
 RNN_project
 Untitled0.ipynb


'/content/drive/MyDrive/keras_nlp_QA_example_1105.ckpt'

In [53]:
model_load = TFAutoModelForQuestionAnswering.from_pretrained(model_checkpoint)
model_load.load_weights(checkpoint_path)

Some layers from the model checkpoint at distilbert-base-cased were not used when initializing TFDistilBertForQuestionAnswering: ['activation_13', 'vocab_transform', 'vocab_layer_norm', 'vocab_projector']
- This IS expected if you are initializing TFDistilBertForQuestionAnswering from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing TFDistilBertForQuestionAnswering from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some layers of TFDistilBertForQuestionAnswering were not initialized from the model checkpoint at distilbert-base-cased and are newly initialized: ['qa_outputs', 'dropout_39']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [46]:
model.save('/content/drive/MyDrive/keras_nlp_QA_example_1105.h5')

NotImplementedError: ignored

In [41]:
# 학습 잘 했다고 가정하고, 질문과 문항이 있다고 하면...

context = """Keras is an API designed for human beings, not machines. Keras follows best
practices for reducing cognitive load: it offers consistent & simple APIs, it minimizes
the number of user actions required for common use cases, and it provides clear &
actionable error messages. It also has extensive documentation and developer guides. """
question = "What is Keras?"

inputs = tokenizer([context], [question], return_tensors="np")
outputs = model(inputs)
start_position = tf.argmax(outputs.start_logits, axis=1)
end_position = tf.argmax(outputs.end_logits, axis=1)
print(int(start_position), int(end_position[0]))
# 로=짓이 가장 큰 위치를 뽑아서 그 해당 위치의 input에서의 단어를 가져와야해서 decoder를 사용.
# 단어의 순서가 26, 12 이ㅓㅀ게 나올 수 있다. 이것도 후처리 해주는 작업이 필요하긴 하다.

31 55


In [42]:
start_position, end_position

(<tf.Tensor: shape=(1,), dtype=int64, numpy=array([31])>,
 <tf.Tensor: shape=(1,), dtype=int64, numpy=array([55])>)

In [43]:
answer = inputs["input_ids"][0, int(start_position) : int(end_position) + 1]
print(answer)

[  117  1122 20220  1116  1103  1295  1104  4795  3721  2320  1111  1887
  1329  2740   117  1105  1122  2790  2330   111  2168  1895  7353  7416
   119]


In [44]:
print(tokenizer.decode(answer))

, it minimizes the number of user actions required for common use cases, and it provides clear & actionable error messages.


In [ ]:
model.push_to_hub("transformers-qa", organization="keras-io")
tokenizer.push_to_hub("transformers-qa", organization="keras-io")

In [ ]:
from huggingface_hub.keras_mixin import push_to_hub_keras

push_to_hub_keras(
    model=model, repo_url="https://huggingface.co/your-username/your-awesome-model"
)
from_pretrained_keras("your-username/your-awesome-model") # load your model